In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import requests
from pandas.io.json import json_normalize
import folium

In [2]:
# The code was removed by Watson Studio for sharing.

In [16]:
LIMIT = 100
countries = ['India', 'Pakistan', 'China', 'USA']
results = {}
for country in countries:
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&near={}&limit={}&categoryId={}'.format(
        CLIENT_ID, 
        CLIENT_SECRET, 
        VERSION, 
        country,
        LIMIT,
        "4e52adeebd41615f56317744")
    results[country] = requests.get(url).json()

In [17]:
df_venues={}
for country in countries:
    venues = json_normalize(results[country]['response']['groups'][0]['items'])
    df_venues[country] = venues[['venue.name', 'venue.location.address', 'venue.location.lat', 'venue.location.lng']]
    df_venues[country].columns = ['Name', 'Address', 'Lat', 'Lng']

In [18]:
maps = {}
for country in countries:
    country_lat = np.mean([results[country]['response']['geocode']['geometry']['bounds']['ne']['lat'],
                        results[country]['response']['geocode']['geometry']['bounds']['sw']['lat']])
    country_lng = np.mean([results[country]['response']['geocode']['geometry']['bounds']['ne']['lng'],
                        results[country]['response']['geocode']['geometry']['bounds']['sw']['lng']])
    maps[country] = folium.Map(location=[country_lat, country_lng], zoom_start=4)

    # adding markers to map
    for lat, lng, label in zip(df_venues[country]['Lat'], df_venues[country]['Lng'], df_venues[country]['Name']):
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color='blue',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(maps[country])  
    print(f"Total number of Military Bases in {country} = ", results[country]['response']['totalResults'])

Total number of Military Bases in India =  18
Total number of Military Bases in Pakistan =  10
Total number of Military Bases in China =  22
Total number of Military Bases in USA =  226


In [13]:
print(f"Total number of Military Bases in {countries[0]} = ", results[countries[0]]['response']['totalResults'])
maps[countries[0]]

Total number of Military Bases in India =  18


In [14]:
print(f"Total number of Military Bases in {countries[1]} = ", results[countries[1]]['response']['totalResults'])
maps[countries[1]]

Total number of Military Bases in Pakistan =  10


In [15]:
print(f"Total number of Military Bases in {countries[2]} = ", results[countries[2]]['response']['totalResults'])
maps[countries[2]]

Total number of Military Bases in China =  22


In [19]:
print(f"Total number of Military Bases in {countries[3]} = ", results[countries[3]]['response']['totalResults'])
maps[countries[3]]

Total number of Military Bases in USA =  226
